In [1]:
import ipl 

organism = ipl.Organism()
organism.verbosity = 1

game = ipl.games.ElMazeGame(3,2)
organism.configure(game.player_config())

action_program = [
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 0, 0]
]


In [2]:
organism.reset_state()
for iturn, pa in enumerate(action_program):
  print('\nRUNNER: Turn {}'.format(iturn + 1))
  organism.handle_sensor_input(game.sensors())

  oa = organism.choose_action(pa)
  if oa.outcomes and len(oa.outcomes):
    print('Expected outcomes:')
    for outcome in oa.outcomes:
      print('\t{}'.format(outcome))
  else:
    print('(Action has no outcomes)')

  game.act(oa.actuators)
  

print()
print('RUNNER: Predefined action sequence complete.')
organism.handle_sensor_input(game.sensors())




RUNNER: Turn 1
ORGANISM: Received sensor input: [1, 0, 0, 0, 0]
ORGANISM: Committing to action: ACTION: [1, 0, 0, 0] (-> 9 outcomes)
Expected outcomes:
	OUTCOME: [1, 0, 0, 0, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 0, 1, 0, 0] (11% $0.00 = $0.00)
	OUTCOME: [0, 1, 1, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 0, 0, 1, 0] (11% $0.00 = $0.00)
	OUTCOME: [0, 1, 0, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 1, 0, 1, 0] (11% $0.00 = $0.00)
	OUTCOME: [1, 0, 1, 1, 1] (11% $1.00 = $0.11)
	OUTCOME: [1, 0, 0, 0, 0] (11% $0.00 = $0.00)
	OUTCOME: [1, 0, 0, 1, 1] (11% $1.00 = $0.11)

RUNNER: Turn 2
ORGANISM: Received sensor input: [1, 0, 0, 1, 0]
ESTIMATOR: Observed vector: [1, 0, 0, 1, 0]  s=1
ESTIMATOR: Counterfactual expected vector: [1, 0, 0, 0, 1]  s=0.60
ESTIMATOR: Counterfactual expected vector: [1, 0, 1, 0, 0]  s=0.60
ESTIMATOR: Counterfactual expected vector: [0, 1, 1, 1, 1]  s=0.20
ESTIMATOR: Counterfactual expected vector: [0, 1, 0, 1, 1]  s=0.40
ESTIMATOR: Counterfactual expected vector: [1, 1, 0

In [3]:
game.io_vector_labels()

{'sensors': ['FORWARD', 'LEFT', 'RIGHT', 'BACK', 'VICTORY'],
 'actuators': ['GO', 'TURN LEFT', 'TURN RIGHT', 'TURN BACK']}

In [36]:
for i in range(0,31):
    a = [int(x) for x in list('{:05b}'.format(i))]
    est = organism.outcome_likelihood_estimator.estimate([0, 1, 0, 1, 0], [0, 1, 0, 0],  a)
    print('{}: {:2d}%'.format(a, int(est*100)))


[0, 0, 0, 0, 0]: 58%
[0, 0, 0, 0, 1]: 39%
[0, 0, 0, 1, 0]: 82%
[0, 0, 0, 1, 1]: 61%
[0, 0, 1, 0, 0]: 38%
[0, 0, 1, 0, 1]: 22%
[0, 0, 1, 1, 0]: 59%
[0, 0, 1, 1, 1]: 41%
[0, 1, 0, 0, 0]: 37%
[0, 1, 0, 0, 1]: 21%
[0, 1, 0, 1, 0]: 58%
[0, 1, 0, 1, 1]: 40%
[0, 1, 1, 0, 0]: 20%
[0, 1, 1, 0, 1]:  7%
[0, 1, 1, 1, 0]: 38%
[0, 1, 1, 1, 1]: 23%
[1, 0, 0, 0, 0]: 79%
[1, 0, 0, 0, 1]: 59%
[1, 0, 0, 1, 0]: 104%
[1, 0, 0, 1, 1]: 82%
[1, 0, 1, 0, 0]: 57%
[1, 0, 1, 0, 1]: 39%
[1, 0, 1, 1, 0]: 80%
[1, 0, 1, 1, 1]: 60%
[1, 1, 0, 0, 0]: 56%
[1, 1, 0, 0, 1]: 38%
[1, 1, 0, 1, 0]: 80%
[1, 1, 0, 1, 1]: 59%
[1, 1, 1, 0, 0]: 36%
[1, 1, 1, 0, 1]: 21%
[1, 1, 1, 1, 0]: 57%


In [17]:
organism.verbosity = 0
for i in range(0, 100):
    print('Run #{}'.format(i))
    game = ipl.games.ElMazeGame(3,2)    

    organism.reset_state()
    for pa in action_program:
      organism.handle_sensor_input(game.sensors())
      oa = organism.choose_action(pa)
      game.act(oa.actuators)


Run #0
Run #1
Run #2
Run #3
Run #4
Run #5
Run #6
Run #7
Run #8
Run #9
Run #10
Run #11
Run #12
Run #13
Run #14
Run #15
Run #16
Run #17
Run #18
Run #19
Run #20
Run #21
Run #22
Run #23
Run #24
Run #25
Run #26
Run #27
Run #28
Run #29
Run #30
Run #31
Run #32
Run #33
Run #34
Run #35
Run #36
Run #37
Run #38
Run #39
Run #40
Run #41
Run #42
Run #43
Run #44
Run #45
Run #46
Run #47
Run #48
Run #49
Run #50
Run #51
Run #52
Run #53
Run #54
Run #55
Run #56
Run #57
Run #58
Run #59
Run #60
Run #61
Run #62
Run #63
Run #64
Run #65
Run #66
Run #67
Run #68
Run #69
Run #70
Run #71
Run #72
Run #73
Run #74
Run #75
Run #76
Run #77
Run #78
Run #79
Run #80
Run #81
Run #82
Run #83
Run #84
Run #85
Run #86
Run #87
Run #88
Run #89
Run #90
Run #91
Run #92
Run #93
Run #94
Run #95
Run #96
Run #97
Run #98
Run #99


In [ ]:
victory_field_idx = game.io_vector_labels()['sensors'].index('VICTORY')
fn_utility = lambda s: s[victory_field_idx]

fn_utility([0,0,1,0,0])